# Docker Engineのインストール

---

構築したOpenHPC環境に[Docker Engine](https://www.docker.com/)をインストールします。

## 前提条件

このNotebookを実行するための前提条件を満たしていることを確認します。

以下のことを前提条件とします。
* 構築済のOpenHPC環境がある
* OpenHPC環境の各ノードに対してAnsibleで操作できるように設定されている

VCノードを作成時に指定した値を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

各ノードに対してAnsibleによる操作が行えることを確認します。操作対象となる UnitGroup 名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name = 

### チェック

指定されたUnitGroupに対して簡単なチェックを行います。

group_varsの値を読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

操作対象となるansibleグループを選択します。計算ノードのスケジュール設定が行われている場合はmasterノードに対して更新を行います。そうでない場合は直接計算ノードに対して設定を行います。

In [ ]:
target = f"{ugroup_name}_compute" if "vcnode_schedule" not in gvars else f"{ugroup_name}_master"

疎通確認を行います。

In [ ]:
!ansible {target} -m ping

管理者として操作できることを確認します。

In [ ]:
!ansible {target} -b -a 'whoami'

## Dockerのインストール

計算ノードにDocker Engineをインストールします。またOpenHPCをGPUの利用向けに構築した場合は[Setting up NVIDIA Container Toolkit](https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/install-guide.html#id2)のインストールをあわせて行います。

dockerなどのインストールを行うplaybookを実行します。スケジュール設定を行なっている場合は直接計算ノードに対して変更を行うのではなくmasterノードで実行しているコンテナからdockerなどのインストールを行います。これは計算ノード数を管理しているコンテナが現時点で起動している計算ノードを把握しているためです。

In [ ]:
!ansible-playbook -Dv -l {target} -e confirm=true playbooks/setup-compute-docker.yml

計算ノードにdockerがインストールされたことを確認します。スケジュール設定を行なっている場合は１ノードだけの結果を確認しています。必要に応じて次のセルで指定している`node_count`の値を変更してください。

In [ ]:
node_count = gvars["compute_nodes"] if "vcnode_schedule" not in gvars else 1
!ansible {ugroup_name}_master -a 'srun -N {node_count} docker info'

GPUを利用する環境では上のセルの実行結果にある`Runtimes`で`nvidia`が表示されていることを確認してください。
表示例を以下に示します。

```
Server:
(中略)
 Runtimes: io.containerd.runc.v2 nvidia runc
 Default Runtime: runc
(以降略) 
```

計算ノードにdockerをインストールしたことをgroup_varsに記録します。

In [ ]:
%run scripts/group.py
update_group_vars(ugroup_name, compute_docker=True)